In [1]:
# Komórka 1: Instalacja i importowanie bibliotek
# W środowisku Jupyter może być konieczne zainstalowanie pycaret
# !pip install python-dotenv boto3 pandas scikit-learn joblib "pycaret[full]"
from dotenv import load_dotenv
import pandas as pd
import boto3
from botocore.client import Config
import os
import joblib
import io
# Importujemy moduł regresji z PyCaret
from pycaret.regression import setup, compare_models, finalize_model, save_model as pycaret_save_model

# Komórka 2: Wczytanie zmiennych środowiskowych
load_dotenv()

# Pobieranie konfiguracji ze zmiennych środowiskowych
access_key = os.getenv('DO_SPACES_KEY')
secret_key = os.getenv('DO_SPACES_SECRET')
endpoint_url = os.getenv('DO_SPACES_ENDPOINT_URL')
bucket_name = os.getenv('DO_SPACES_BUCKET')


# Komórka 3: Konfiguracja klienta S3 (DigitalOcean Spaces)
# --- POPRAWKA ---
# Zmieniamy styl adresowania na 'path', który jest bardziej niezawodny dla
# S3-kompatybilnych usług jak DigitalOcean Spaces. To rozwiązuje błąd SSL.
try:
    region_name = endpoint_url.split('//')[1].split('.')[0]
except (IndexError, AttributeError):
    print("BŁĄD: Nieprawidłowy format DO_SPACES_ENDPOINT_URL. Sprawdź plik .env.")
    region_name = None

session = boto3.session.Session()
client = session.client('s3',
                        config=Config(s3={'addressing_style': 'path'}), # <-- KLUCZOWA ZMIANA
                        region_name=region_name,
                        endpoint_url=endpoint_url,
                        aws_access_key_id=access_key,
                        aws_secret_access_key=secret_key)

print(f"Połączono z DigitalOcean Spaces w regionie: {region_name}")

# Komórka 4: Funkcja do wczytywania danych ze Spaces
def load_data_from_spaces(file_keys):
    """Wczytuje wiele plików CSV ze Spaces i łączy je w jeden DataFrame."""
    all_dfs = []
    for key in file_keys:
        try:
            response = client.get_object(Bucket=bucket_name, Key=key)
            csv_content = response['Body'].read()
            df = pd.read_csv(io.BytesIO(csv_content), sep=';')
            all_dfs.append(df)
            print(f"Pomyślnie wczytano plik: {key}")
        except Exception as e:
            print(f"Błąd podczas wczytywania pliku {key}: {e}")
            return None
    if not all_dfs:
        return pd.DataFrame()
    return pd.concat(all_dfs, ignore_index=True)

# Komórka 5: Funkcja do konwersji czasu na sekundy
def time_to_seconds(time_str):
    """Konwertuje czas w formacie HH:MM:SS lub MM:SS na sekundy."""
    if pd.isna(time_str):
        return None
    parts = str(time_str).split(':')
    if len(parts) == 3:
        h, m, s = map(int, parts)
        return h * 3600 + m * 60 + s
    elif len(parts) == 2:
        m, s = map(int, parts)
        return m * 60 + s
    return None

# Komórka 6: Czyszczenie i przygotowanie danych
def clean_and_prepare_data(df):
    """Czyści i przygotowuje dane do trenowania modelu."""
    print("Rozpoczynam czyszczenie danych...")
    
    # --- POPRAWKA BŁĘDU 'KeyError' ---
    # Zdefiniuj nazwy kolumn, których potrzebujesz, na podstawie danych z pliku.
    # Zwróć uwagę na wielkość liter, spacje i polskie znaki.
    original_cols = ['Płeć', 'Rocznik', '5 km Czas', 'Czas']
    
    # Zdefiniuj nowe, proste nazwy dla tych kolumn.
    new_cols_map = {
        'Płeć': 'plec',
        'Rocznik': 'wiek',
        '5 km Czas': 'czas_5km',
        'Czas': 'czas_netto'
    }
    
    try:
        # Sprawdzenie, czy wszystkie potrzebne kolumny istnieją w DataFrame
        if not all(col in df.columns for col in original_cols):
             missing = [col for col in original_cols if col not in df.columns]
             print(f"\nBŁĄD KRYTYCZNY: Brak następujących kolumn w pliku CSV: {missing}")
             return None

        # Wybierz tylko potrzebne kolumny
        df_clean = df[original_cols].copy()
        
        # Zmień nazwy kolumn na prostsze, angielskie odpowiedniki
        df_clean.rename(columns=new_cols_map, inplace=True)
        
    except KeyError as e:
        print(f"\nBŁĄD KRYTYCZNY: Nie znaleziono wymaganych kolumn w danych! {e}")
        return None
        
    # Oblicz wiek na podstawie rocznika. Zakładamy, że wyścig odbył się w 2024 roku.
    # Można to ulepszyć, np. pobierając rok z nazwy pliku.
    current_year = 2024 
    df_clean['wiek'] = current_year - df_clean['wiek']
        
    df_clean['czas_netto_s'] = df_clean['czas_netto'].apply(time_to_seconds)
    df_clean['czas_5km_s'] = df_clean['czas_5km'].apply(time_to_seconds)
    df_clean.dropna(subset=['czas_netto_s', 'czas_5km_s', 'wiek', 'plec'], inplace=True)
    df_clean['tempo_5km_s_na_km'] = df_clean['czas_5km_s'] / 5
    df_clean.drop(['czas_netto', 'czas_5km', 'czas_5km_s'], axis=1, inplace=True)
    df_clean['wiek'] = df_clean['wiek'].astype(int)
    print(f"Czyszczenie zakończone. Pozostało {len(df_clean)} wierszy.")
    print("Przykładowe dane po czyszczeniu:")
    print(df_clean.head())
    return df_clean

# Komórka 7: Trenowanie i wybór najlepszego modelu za pomocą PyCaret
def train_and_select_model(df):
    """Używa PyCaret do automatycznego znalezienia i wytrenowania najlepszego modelu."""
    print("Rozpoczynam proces AutoML z PyCaret...")
    
    s = setup(data=df, 
              target='czas_netto_s', 
              categorical_features=['plec'],
              numeric_features=['wiek', 'tempo_5km_s_na_km'],
              session_id=123,
              verbose=False,
              log_experiment=False
             )
    
    print("Porównuję modele, aby znaleźć najlepszy...")
    best_model = compare_models(sort='MAE')
    
    print("\nWyniki porównania modeli:")
    print(best_model)
    
    final_model = finalize_model(best_model)
    
    print("\nSfinalizowany najlepszy model:")
    print(final_model)
    
    return final_model

# Komórka 8: Funkcja do zapisywania modelu
def save_and_upload_model(model, local_path, remote_path):
    """Zapisuje model lokalnie i wysyła do DigitalOcean Spaces."""
    pycaret_save_model(model, local_path)
    print(f"Model zapisany lokalnie w pliku: {local_path}.pkl")
    
    try:
        with open(f"{local_path}.pkl", 'rb') as f:
            client.upload_fileobj(f, bucket_name, remote_path)
        print(f"Model pomyślnie wysłany do DigitalOcean Spaces jako: {remote_path}")
    except Exception as e:
        print(f"Błąd podczas wysyłania modelu do Spaces: {e}")

# Komórka 9: Główna logika pipeline'u
if __name__ == '__main__':
    files = ['halfmarathon_wroclaw_2023__final.csv', 'halfmarathon_wroclaw_2024__final.csv']
    raw_data = load_data_from_spaces(files)
    
    if raw_data is not None and not raw_data.empty:
        cleaned_data = clean_and_prepare_data(raw_data)
        
        # Sprawdzamy, czy czyszczenie danych się powiodło
        if cleaned_data is not None and not cleaned_data.empty:
            best_pipeline = train_and_select_model(cleaned_data)
            
            local_model_path_no_ext = 'halfmarathon_pipeline'
            remote_model_path = 'models/halfmarathon_pipeline.pkl'
            
            save_and_upload_model(best_pipeline, local_model_path_no_ext, remote_model_path)
            
            print("\nPipeline AutoML zakończony sukcesem!")
        else:
            print("\nZatrzymano pipeline z powodu błędu podczas czyszczenia danych.")


Połączono z DigitalOcean Spaces w regionie: fra1
Pomyślnie wczytano plik: halfmarathon_wroclaw_2023__final.csv
Pomyślnie wczytano plik: halfmarathon_wroclaw_2024__final.csv
Rozpoczynam czyszczenie danych...
Czyszczenie zakończone. Pozostało 17927 wierszy.
Przykładowe dane po czyszczeniu:
  plec  wiek  czas_netto_s  tempo_5km_s_na_km
0    M    32        3899.0              175.4
1    M    38        3983.0              177.6
2    M    28        4104.0              189.2
3    M    36        4216.0              194.2
4    M    29        4227.0              194.4
Rozpoczynam proces AutoML z PyCaret...
Porównuję modele, aby znaleźć najlepszy...


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,952.4740,1475472.4737,1214.2553,-0.0066,0.1628,0.1305,0.1070
br,Bayesian Ridge,958.0172,1466835.0870,1210.7256,-0.0008,0.1633,0.1329,0.0390
dummy,Dummy Regressor,958.0479,1467071.5500,1210.8201,-0.0009,0.1633,0.1329,0.0360
omp,Orthogonal Matching Pursuit,958.1244,1467050.0487,1210.8141,-0.0009,0.1633,0.1329,0.0560
en,Elastic Net,958.1336,1467109.8198,1210.8412,-0.0010,0.1633,0.1329,0.0430
lasso,Lasso Regression,958.2577,1467442.5510,1210.9822,-0.0012,0.1633,0.1329,0.0430
llar,Lasso Least Angle Regression,958.2577,1467442.5527,1210.9822,-0.0012,0.1633,0.1329,0.0430
ridge,Ridge Regression,958.2951,1467532.5331,1211.0198,-0.0013,0.1634,0.1329,0.0410
lar,Least Angle Regression,958.2953,1467532.8597,1211.0199,-0.0013,0.1634,0.1329,0.0380
lr,Linear Regression,958.2953,1467532.8597,1211.0199,-0.0013,0.1634,0.1329,1.0340


Processing:   0%|          | 0/77 [00:00<?, ?it/s]


Wyniki porównania modeli:
HuberRegressor()

Sfinalizowany najlepszy model:
Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['wiek', 'tempo_5km_s_na_km'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['plec'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 TransformerWrapper(include=['plec'],
                                    transformer=OrdinalEncoder(cols=['plec'],
                                                               handle_missing='return_nan',
                                                               mapping=[{'col': 'plec',
                                                                         'data_type': dtype('O'),
                                                                         